In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score , auc
from sklearn.model_selection import train_test_split

import cv2
from PIL import Image
from tensorflow import keras
from keras import Sequential
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_hub as hub

from keras.applications import MobileNetV2
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D

In [3]:
train_path = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_path = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

In [5]:
import tensorflow as tf
train_data=tf.keras.preprocessing.image_dataset_from_directory(
train_path,
    batch_size=256,
    image_size=(224,224),shuffle=True
)

Found 70295 files belonging to 38 classes.


In [6]:
valid_data = tf.keras.preprocessing.image_dataset_from_directory(
 valid_path,
    batch_size=256,
  image_size=(224,224), shuffle=True
)

Found 17572 files belonging to 38 classes.


In [7]:
path = "/kaggle/input/new-plant-diseases-dataset/test/test"
test_folder=os.listdir(path)
test_folder[:5]

['TomatoEarlyBlight6.JPG',
 'TomatoYellowCurlVirus4.JPG',
 'TomatoYellowCurlVirus6.JPG',
 'PotatoHealthy2.JPG',
 'TomatoYellowCurlVirus5.JPG']

In [9]:
len(train_data.class_names)

38

In [10]:
class_labels=train_data.class_names
class_labels

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [11]:
datagen=ImageDataGenerator(rescale=1.0/255.0)
train_scaled_data = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=256,shuffle=True,
    class_mode='sparse'
)

# test_scaled_data = datagen.flow_from_directory(
#     "/kaggle/input/100-bird-species/test",
#     target_size=(224, 224),
#     batch_size=256,shuffle=True
# )

valid_scaled_data = datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=256,shuffle=True,
    class_mode='sparse'
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [12]:
train_scaled_data.class_indices

{'Apple___Apple_scab': 0,
 'Apple___Black_rot': 1,
 'Apple___Cedar_apple_rust': 2,
 'Apple___healthy': 3,
 'Blueberry___healthy': 4,
 'Cherry_(including_sour)___Powdery_mildew': 5,
 'Cherry_(including_sour)___healthy': 6,
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7,
 'Corn_(maize)___Common_rust_': 8,
 'Corn_(maize)___Northern_Leaf_Blight': 9,
 'Corn_(maize)___healthy': 10,
 'Grape___Black_rot': 11,
 'Grape___Esca_(Black_Measles)': 12,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13,
 'Grape___healthy': 14,
 'Orange___Haunglongbing_(Citrus_greening)': 15,
 'Peach___Bacterial_spot': 16,
 'Peach___healthy': 17,
 'Pepper,_bell___Bacterial_spot': 18,
 'Pepper,_bell___healthy': 19,
 'Potato___Early_blight': 20,
 'Potato___Late_blight': 21,
 'Potato___healthy': 22,
 'Raspberry___healthy': 23,
 'Soybean___healthy': 24,
 'Squash___Powdery_mildew': 25,
 'Strawberry___Leaf_scorch': 26,
 'Strawberry___healthy': 27,
 'Tomato___Bacterial_spot': 28,
 'Tomato___Early_blight': 29,
 'Toma

In [13]:
for img,label in train_scaled_data:
    print(label)
    break

[27.  4. 15. 13. 34. 19. 19. 16. 21. 19.  3. 14. 11.  8. 27.  9. 19.  7.
 32.  3. 32. 22. 32. 13. 16.  6. 36. 16.  1. 23.  7. 32.  9. 21. 33. 25.
 32.  1. 12.  8.  8. 26.  3. 23. 32. 13.  8. 28. 28. 19. 15. 14. 20. 33.
  0. 13.  3. 13. 17. 23. 31. 23. 30. 19. 28. 25. 27. 20. 33.  2. 34. 26.
 31. 28. 19. 17. 21.  5. 19. 19. 27. 15. 13. 27. 18. 27. 17. 17. 35. 27.
  2. 30.  7.  3. 34.  0. 33. 16.  3. 10. 34.  5.  8. 20.  5. 37. 18. 21.
 23. 19. 22. 24. 34. 16. 15. 25. 18. 15. 24. 16. 34. 20. 24. 34. 29.  0.
 21. 22. 16. 34.  1.  7. 14. 29. 21. 29. 30.  2.  6.  2. 14. 19. 17. 33.
 15. 27. 19.  2. 32.  2. 12. 28. 21.  6. 18. 18. 36. 26. 19. 31. 24. 24.
  2.  2. 33. 19. 18. 33. 14. 36.  8. 23.  6. 10. 33. 11. 14. 11. 10. 13.
 14.  6. 32. 14. 15.  8. 26. 19. 33. 25. 23. 13. 19. 34. 11. 15. 13. 35.
  8. 21. 18. 27. 30. 27. 15.  6.  8. 15.  8.  0. 31. 16. 35. 11. 25.  2.
 12. 15. 33. 16. 37. 32.  8. 35. 13. 29. 31.  0. 24. 34. 34. 35. 29. 22.
 15. 11.  1. 27. 37. 30.  0. 16. 37.  0. 12.  8. 12

In [14]:
label[10]

3.0

In [16]:
base_model=MobileNetV2(input_shape=(224,224,3),
                      include_top=False,weights='imagenet')

for layer in base_model.layers:
    layer.trainable=False
    
model=Sequential()

model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(len(class_labels),activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 100)               128100    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 38)                3838      
                                                           

In [19]:
history = model.fit(train_scaled_data ,batch_size=256,epochs=5,
                verbose=1,
                validation_data=valid_scaled_data)

Epoch 1/5
275/275 [==============================] - 671s 2s/step - loss: 0.2673 - accuracy: 0.9167 - val_loss: 0.2059 - val_accuracy: 0.9352
Epoch 2/5
275/275 [==============================] - 320s 1s/step - loss: 0.1652 - accuracy: 0.9470 - val_loss: 0.1733 - val_accuracy: 0.9429
Epoch 3/5
275/275 [==============================] - 300s 1s/step - loss: 0.1246 - accuracy: 0.9589 - val_loss: 0.1483 - val_accuracy: 0.9494
Epoch 4/5
275/275 [==============================] - 298s 1s/step - loss: 0.1004 - accuracy: 0.9668 - val_loss: 0.1310 - val_accuracy: 0.9574
Epoch 5/5
275/275 [==============================] - 298s 1s/step - loss: 0.0847 - accuracy: 0.9726 - val_loss: 0.1489 - val_accuracy: 0.9503


In [22]:
loss,acc = model.evaluate(train_scaled_data)
print("Loss on Train data:",loss)
print("Accuracy on Train data:",acc)

loss1,acc1 = model.evaluate(valid_scaled_data)

print("Loss on Test data:",loss1)
print("Accuracy on Test data:",acc1)

275/275 [==============================] - 235s 853ms/step - loss: 0.0807 - accuracy: 0.9741
Loss on Train data: 0.080684132874012
Accuracy on Train data: 0.9741233587265015
69/69 [==============================] - 60s 869ms/step - loss: 0.1489 - accuracy: 0.9503
Loss on Test data: 0.14894069731235504
Accuracy on Test data: 0.9502617716789246
